In [ ]:
import json
import os
import pickle
import sys

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from clang_interop.cl_types import ClContext
from clang_interop.process_clang_output import process_clang_output

sys.path.append("../autoware/build/tracetools_read/")
sys.path.append("../autoware/build/tracetools_analysis/")
from tracetools_read.trace import *
from tracetools_analysis.loading import load_file
from tracetools_analysis.processor.ros2 import Ros2Handler
from tracetools_analysis.utils.ros2 import Ros2DataModelUtil

from dataclasses import dataclass
from typing import List, Dict, Set, Tuple

from tracing_interop.tr_types import TrTimer, TrTopic, TrPublisher, TrPublishInstance, TrCallbackInstance, \
TrCallbackSymbol, TrCallbackObject, TrSubscriptionObject, TrContext
from misc.utils import ProgressPrinter, cached

In [ ]:
TR_PATH = os.path.expanduser("data/trace-awsim-x86/ust")
CL_PATH = os.path.expanduser("~/Projects/llvm-project/clang-tools-extra/ros2-internal-dependency-checker/output")

# Organize Trace Data

In [ ]:
def _load_traces():
    file = load_file(TR_PATH)
    handler = Ros2Handler.process(file)
    util = Ros2DataModelUtil(handler)
    return TrContext(util, handler)


_tracing_context = cached("tr_objects", _load_traces, [TR_PATH])
_tr_globals = ["nodes", "publishers", "subscriptions", "timers", "timer_node_links", "subscription_objects",
               "callback_objects", "callback_symbols", "publish_instances", "callback_instances", "topics"]

# Help the IDE recognize those identifiers
nodes = publishers = subscriptions = timers = timer_node_links = subscription_objects = callback_objects = callback_symbols = publish_instances = callback_instances = topics = None

for name in _tr_globals:
    globals()[name] = getattr(_tracing_context, name)

print("Done.")


# ROS2 Tracing & Clang Matching

In [ ]:
def _load_cl_objects():
    return process_clang_output(CL_PATH)


_cl_context: ClContext = cached("cl_objects", _load_cl_objects, [CL_PATH])

In [ ]:
from matching.subscriptions import cl_deps_to_tr_deps, match

matches, tr_unmatched, cl_unmatched = match(_tracing_context, _cl_context)
tr_internal_deps = cl_deps_to_tr_deps(matches, _tracing_context, _cl_context)
tr_cl_matches = {tup[1]: tup[0] for tup in matches}

print(len(tr_internal_deps), sum(map(len, tr_internal_deps.values())))

# E2E Latency Calculation

In [ ]:
from latency_graph import latency_graph as lg

#lat_graph = lg.LatencyGraph(_tracing_context)

import pickle

#with open("lat_graph.pkl", "wb") as f:
#    pickle.dump(lat_graph, f)
with open("lat_graph.pkl", "rb") as f:
    lat_graph = pickle.load(f)

In [ ]:
from matching.subscriptions import sanitize
from typing import Iterable, Sized
from tracing_interop.tr_types import TrNode, TrCallbackObject, TrCallbackSymbol, TrSubscriptionObject

#################################################
# Plot DFG
#################################################

# Compare with: https://autowarefoundation.github.io/autoware-documentation/main/design/autoware-architecture/node-diagram/
node_colors = {
        "sensing": {"fill": "#e1d5e7", "stroke": "#9673a6"},
        "localization": {"fill": "#dae8fc", "stroke": "#6c8ebf"},
        "perception": {"fill": "#d5e8d4", "stroke": "#82b366"},
        "planning": {"fill": "#fff2cc", "stroke": "#d6b656"},
        "control": {"fill": "#ffe6cc", "stroke": "#d79b00"},
        "system": {"fill": "#f8cecc", "stroke": "#b85450"},
        "vehicle_interface": {"fill": "#b0e3e6", "stroke": "#0e8088"},
        None: {"fill": "#f5f5f5", "stroke": "#666666"}
}

node_namespace_mapping = {
        'perception': 'perception',
        'sensing': 'sensing',
        'planning': 'planning',
        'control': 'control',
        'awapi': None,
        'autoware_api': None,
        'map': None,
        'system': 'system',
        'localization': 'localization',
        'robot_state_publisher': None,
        'aggregator_node': None,
        'pointcloud_container': 'sensing',
}

import graphviz as gv

g = gv.Digraph('G', filename="latency_graph.gv",
               node_attr={'shape': 'plain'},
               graph_attr={'pack': '1'})
g.graph_attr['rankdir'] = 'LR'

def plot_hierarchy(gv_parent, lg_node: lg.LGHierarchyLevel, **subgraph_kwargs):
    if lg_node.name == "[NONE]":
        return

    print(f"{'  ' * lg_node.full_name.count('/')}Processing {lg_node.name}: {len(lg_node.callbacks)}")
    with gv_parent.subgraph(name=f"cluster_{lg_node.full_name.replace('/', '__')}", **subgraph_kwargs) as c:
        c.attr(label=lg_node.name)
        for cb in lg_node.callbacks:
            if isinstance(cb, lg.LGTrCallback):
                tr_cb = cb.cb
                try:
                    sym = _tracing_context.callback_symbols.get(tr_cb.callback_object)
                    pretty_sym = repr(sanitize(sym.symbol)).replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")
                except KeyError:
                    pretty_sym = cb.name
                except TypeError:
                    pretty_sym = cb.name
            else:
                pretty_sym = cb.name
            c.node(cb.id(),
                   f'<<table BORDER="0" CELLBORDER="1" CELLSPACING="0"><tr><td port="in"></td><td>{pretty_sym}</td><td port="out"></td></tr></table>>')

        for ch in lg_node.children:
            plot_hierarchy(c, ch, **subgraph_kwargs)

def plot_lg(graph: lg.LatencyGraph):
    for top_level_node in graph.top_node.children:
        colors = node_colors[node_namespace_mapping.get(top_level_node.name)]
        plot_hierarchy(g, top_level_node, graph_attr={'bgcolor': colors["fill"], 'pencolor': colors["stroke"]})

    for edge in graph.edges:
        g.edge(f"{edge.start.id()}:out", f"{edge.end.id()}:in")

plot_lg(lat_graph)

g.save("latency_graph.gv")
g.render("latency_graph.svg")

g

In [ ]:
##################################################
# Compute in/out topics for hierarchy level X
##################################################

HIER_LEVEL = 2

def get_nodes_on_level(lat_graph: lg.LatencyGraph):
    def _traverse_node(node: lg.LGHierarchyLevel, cur_lvl=0):
        if cur_lvl == HIER_LEVEL:
            return [node]

        if not node.children and cur_lvl < HIER_LEVEL:
            return [node]

        collected_nodes = []
        for ch in node.children:
            collected_nodes += _traverse_node(ch, cur_lvl + 1)
        return collected_nodes

    return _traverse_node(lat_graph.top_node)

lvl_nodes = get_nodes_on_level(lat_graph)
lvl_nodes = [n for n in lvl_nodes if "transform_listener_impl" not in n.full_name]

print(', '.join(map(lambda n: n.full_name, lvl_nodes)))

def _collect_callbacks(n: lg.LGHierarchyLevel):
    callbacks = []
    callbacks += n.callbacks
    for ch in n.children:
        callbacks += _collect_callbacks(ch)
    return callbacks


cb_to_node_map = {}
for n in lvl_nodes:
    cbs = _collect_callbacks(n)
    for cb in cbs:
        cb_to_node_map[cb.id()] = n


edges_between_nodes = {}
for edge in lat_graph.edges:
    from_node = cb_to_node_map.get(edge.start.id())
    to_node = cb_to_node_map.get(edge.end.id())

    if from_node is None or to_node is None:
        continue

    if from_node.full_name == to_node.full_name:
        continue

    k = (from_node.full_name, to_node.full_name)

    if k not in edges_between_nodes:
        edges_between_nodes[k] = 0

    edges_between_nodes[k] += 1

g = gv.Digraph('G', filename="latency_graph.gv",
               node_attr={'shape': 'plain'},
               graph_attr={'pack': '1'})
g.graph_attr['rankdir'] = 'LR'

for n in lvl_nodes:
    colors = node_colors[node_namespace_mapping.get(n.full_name.strip("/").split("/")[0])]
    g.node(n.full_name, label=n.full_name, fillcolor=colors["fill"], color=colors["stroke"], shape="box", style="filled")

for (src_name, dst_name), cnt in edges_between_nodes.items():
    print(src_name, dst_name, cnt)
    g.edge(src_name, dst_name, weight=str(cnt))

g.save("level_graph.gv")
g.render("level_graph.svg")

g